# Getting started with XGI
<div>
<img src="../logo/logo.svg" width="200"/>
</div>

XGI is a Python library to make working with and analyzing complex systems with higher-order interactions easy. We have collected a lot of useful functions, algorithms, and tools for working with hypergraphs and simplicial complexes to make life easier.

Main contributors: Nicholas Landry, Leo Torres, Maxime Lucas, Iacopo Iacopini, Giovanni Petri, Alice Patania, and Alice Schwarze.

We will
1. Create and load hypergraphs
2. Visualize hypergraphs and simplicial complexes
3. Show how to use the stats interface
4. Give an example of how to compare an empirical dataset to a null model.

We start off by loading the XGI library.

In [ ]:
import xgi
import numpy as np
import pandas as pd
from IPython.display import display
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt

### Creating a hypergraph

We want to start with a hypergraph, and this can be done in several ways:
1. Build a hypergraph node-by-node and edge-by-edge (less common, but can be helpful in writing your own generative models)
2. Load an existing dataset
3. Sample from a random generative model

Let's start with the first method.

#### Building a hypergraph

In [ ]:
H_build = xgi.Hypergraph()
H_build.add_edge([1, 2], id="a")
H_build.add_node(0)
H_build.add_edges_from([[3, 4], [0, 2, 3]])
H_build.add_nodes_from([9, 10])

XGI automatically assigns unique edge IDs (if a user doesn't specify the ID)

In [ ]:
H_build.edges

Why NodeViews and EdgeViews? These allow users to access many different properties and data structures from nodes and edges in a much simpler way. We will cover this more in depth later. For now, we can get the edges of which each node is a part and the nodes in each edge as follows:

In [ ]:
print(H_build.nodes.memberships())
print(H_build.edges.members())
print(H_build.nodes.memberships(2))
print(H_build.edges.members("a"))

#### Loading datasets

Moving on to method 2, one can load datasets in several different ways. First, we provide a companion data repository, [xgi-data](https://gitlab.com/complexgroupinteractions/xgi-data), where users can easily load several datasets in standard format:

In [ ]:
H_enron = xgi.load_xgi_data("email-enron")

This dataset, for example, has a corresponding [datasheet](https://gitlab.com/complexgroupinteractions/xgi-data-email-enron/-/blob/main/README.md) explaining its characteristics. The nodes (individuals) in this dataset contain associated email addresses and the edges (emails) contain associated timestamps. These attributes can be accessed by simply typing `H.nodes[id]` or `H.edges[id]` respectively.

In [ ]:
print(f"The hypergraph has {H_enron.num_nodes} nodes and {H_enron.num_edges} edges")

In [ ]:
print("The first 10 node IDs are:")
print(list(H_enron.nodes)[:10])
print("\nThe first 10 edge IDs are:")
print(list(H_enron.edges)[:10])
print("\nThe attributes of node '4' are")
print(H_enron.nodes["4"])
print("\nThe attributes of edge '6' are")
print(H_enron.edges["6"])

We can clean up this dataset to remove isolates, multi-edges, singletons, and to replace all IDs with integer IDs.

In [ ]:
H_enron_cleaned = H_enron.cleanup(
    multiedges=False, singletons=False, isolates=False, relabel=True, in_place=False
)

print(
    f"The hypergraph has {H_enron_cleaned.num_nodes} nodes and {H_enron_cleaned.num_edges} edges"
)

print("The first 10 node IDs are:")
print(list(H_enron_cleaned.nodes)[:10])

Likewise, one can load from a file. In the `readwrite` module, there are functions for handling a variety of file formats. We may also be interested in the *dual* hypergraph formed by considering nodes to be genes and edges to be diseases:

In [ ]:
D = H_enron.dual()
print(f"The hypergraph has {D.num_nodes} nodes and {D.num_edges} edges")

#### Random null models

Lastly, we can create  synthetic hypergraphs using random generative models. For example, we can extract the degree and edge size sequence from this dataset and wire them together at random (according to the Chung-Lu model) to create a random null model:

In [ ]:
k = D.nodes.degree.asdict()
s = D.edges.size.asdict()

H_random = xgi.chung_lu_hypergraph(k, s)

We check whether this new hypergraph is connected and if not, the sizes of the connected components:

In [ ]:
connected = xgi.is_connected(H_random)
if not connected:
    size, num = np.unique(
        [len(cc) for cc in xgi.connected_components(H_random)], return_counts=True
    )
    print(f"The connected components:")
    display(pd.DataFrame([size, num], columns=["Component size", "Number"]))

We can do much more such as computing different properties, reading/writing, converting to/from different data structures, hypergraph null models and much more!

See the Read The Docs for more information: https://xgi.readthedocs.io

### Visualization

The first step for drawing a hypergraph is to choose a layout for the nodes. At the moment the available layouts are:
* ```random_layout```: positions nodes uniformly at random in the unit square.
* ```pairwise_spring_layout```: positions the nodes using the Fruchterman-Reingold force-directed algorithm on the projected graph.
* ```barycenter_spring_layout``` and ```weighted_barycenter_spring_layout```: slight modification of ```pairwise_spring_layout```
    
Each layout returns a dictionary that maps nodes ID into (x, y) coordinates.

In [ ]:
seed = 0
H_viz = xgi.random_hypergraph(20, [0.08, 0.01, 0.001], seed=seed)
pos1 = xgi.random_layout(H_viz)
pos2 = xgi.pairwise_spring_layout(H_viz)
pos3 = xgi.barycenter_spring_layout(H_viz)
pos4 = xgi.weighted_barycenter_spring_layout(H_viz)

In [ ]:
xgi.draw(H_viz, pos1)

**Colors of the hyperedges** are designed to match the hyperedge size, but the colormap can be tweaked:

In [ ]:
# Sequential colormap
cmap = cm.Paired

xgi.draw(H_viz, pos4, edge_fc_cmap=cmap)

Other parameters can be changed as well:

In [ ]:
cmap = cm.Reds
dyad_lc = "gray"
dyad_lw = 4
node_fc = "black"
node_ec = "white"
node_lw = 2
node_size = 20

xgi.draw(
    H_viz,
    pos4,
    edge_fc_cmap=cmap,
    dyad_lc=dyad_lc,
    dyad_lw=dyad_lw,
    node_fc=node_fc,
    node_ec=node_ec,
    node_lw=node_lw,
    node_size=node_size,
)

Simplicial complexes can be visualized using the same functions for node layout and drawing.

#### Technical note
By definition, a simplicial complex object contains all sub-simplices. This would make the visualization heavy since all sub-simplices contained in a maximal simplex would overlap. The automatic solution for this, implemented by default in all the layouts, is to convert the simplicial complex into a hypergraph composed by solely by its maximal simplices.

#### Visual note
To visually distinguish simplicial complexes from hypergraphs, the ```draw``` function will also show all links contained in each maximal simplices (while omitting simplices of intermediate orders). 

In [ ]:
SC = xgi.SimplicialComplex()
SC.add_simplices_from([[3, 4, 5], [3, 6], [6, 7, 8, 9], [1, 4, 10, 11, 12], [1, 4]])

pos = xgi.pairwise_spring_layout(SC)
xgi.draw(SC, pos)

We can even color/draw the nodes and edges based on statistics!

In [ ]:
n = 100
is_connected = False
while not is_connected:
    H = xgi.random_hypergraph(n, [0.03, 0.0002, 0.00001])
    is_connected = xgi.is_connected(H)
pos = xgi.barycenter_spring_layout(H)

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.subplot(111)
xgi.draw(
    H,
    pos,
    node_size=H.nodes.degree,
    node_lw=H.nodes.average_neighbor_degree,
    node_fc=H.nodes.degree,
    ax=ax,
)

### The stats package

The stats package is one of the features that sets `xgi` apart from other libraries.  It
provides a common interface to all statistics that can be computed from a network, its
nodes, or edges.

Consider the degree of the nodes of a hypergraph `H`.  After computing the values of the
degrees, one may wish to store them in a dict, a list, an array, a dataframe, etc.
Through the stats package, `xgi` provides a simple interface that seamlessly allows for
this type conversion.  This is done via the `NodeStat` class.

In [ ]:
import xgi
import numpy as np
import pandas as pd

H_stats = xgi.Hypergraph([[1, 2, 3], [2, 3, 4, 5], [3, 4, 5]])
H_stats.degree()

In [ ]:
H_stats.nodes.degree

This `NodeStat` object allows us to choose the datatype, specify keywords, and get basic statistics from these properties

In [ ]:
print("As a dictionary:")
print(H_stats.nodes.degree.asdict())
print("\nAs a list:")
print(H_stats.nodes.degree.aslist())
print("\nAs a numpy array:")
print(H_stats.nodes.degree.asnumpy())
print("\nAs a pandas dataframe:")
print(H_stats.nodes.degree.aspandas())

In [ ]:
H_stats.nodes.degree.aspandas().plot()

In [ ]:
print(H_stats.nodes.degree(order=2).asdict())

In [ ]:
st = H_stats.nodes.degree
np.round([st.max(), st.min(), st.mean(), st.median(), st.var(), st.std()], 3)

Likewise, for edges,

In [ ]:
st = H_stats.edges.size
st

In [ ]:
np.round([st.max(), st.min(), st.mean(), st.median(), st.var(), st.std()], 3)

The interface for attributes is very similar. If we add nodal attributes, for example

In [ ]:
H_stats.add_nodes_from(
    [
        (1, {"color": "red", "name": "horse"}),
        (2, {"color": "blue", "name": "pony"}),
        (3, {"color": "yellow", "name": "zebra"}),
        (4, {"color": "red", "name": "orangutan", "age": 20}),
        (5, {"color": "blue", "name": "fish", "age": 2}),
    ]
)
print(H_stats.nodes.attrs("color").asdict())

In [ ]:
print(H_stats.nodes.attrs[1])

One can also filter nodes and edges by their attributes as well as any associated statistic. For example,

In [ ]:
print(H_stats.nodes.filterby("degree", 2))
print(H_stats.nodes.filterby_attr("color", "blue"))

In [ ]:
H_stats.edges.filterby("size", 3).members()

In [ ]:
H_stats.nodes.multi(["degree", "clustering"]).aspandas().groupby("degree").agg("mean")

In [ ]:
(
    H_stats.nodes.multi(["degree", "clustering", "average_neighbor_degree"])
    .aspandas()
    .groupby("degree")
    .agg("mean")
    .plot(marker="o")
)

For even more functionality, see the Read The Docs: https://xgi.readthedocs.io

## An example

Let's compare the assortativity of an empirical dataset (Let's choose the `email-enron` dataset) to a random null model. We start by loading the Enron dataset and constructing a Chung-Lu hypergraph using the dataset's degree sequence and edge size sequence:

In [ ]:
H = H_enron
k = H.nodes.degree.asdict()
s = H.edges.size.asdict()

H_null = xgi.chung_lu_hypergraph(k, s)

Now we use the definitions of assortativity in "Configuration models of random hypergraphs" by Phil Chodrow to compare the assortativity of the random null model to the original dataset.

In [ ]:
labels = ["uniform", "top-2", "top-bottom"]

assort_orig = []
assort_null = []
for l in labels:
    assort_orig.append(xgi.degree_assortativity(H, kind=l))
    assort_null.append(xgi.degree_assortativity(H_null, kind=l))

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width / 2, assort_orig, width, label="Original")
rects2 = ax.bar(x + width / 2, assort_null, width, label="Null model")
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize=16)
ax.set_ylabel("Assortativity", fontsize=16)
ax.legend()
plt.tight_layout()
plt.show()

From this plot we can see that the original dataset is more assortative than the random null model.

### Dynamics

We can also simulate the order parameter of the Kuramoto model for hypergraphs.

In [ ]:
n = 100
H = xgi.random_hypergraph(n, [0.05, 0.001], seed=None)
w = 2 * np.random.normal(1, 0.05, n)
theta = np.linspace(0, 2 * np.pi, n)
timesteps = 1000
dt = 0.002
R = xgi.compute_kuramoto_order_parameter(
    H, k2=2, k3=3, w=w, theta=theta, timesteps=timesteps, dt=dt
)

In [ ]:
plt.figure()
plt.plot(np.arange(0, timesteps * dt, dt), R)
plt.xlabel("time")
plt.ylabel("Kuramoto order parameter")
plt.show()

# For more details, visit https://xgi.readthedocs.io